In [56]:
#Cargar las bibliotecas
import pandas as pd
import ast
#Cargar los datos
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/datasets/electronics_store.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [58]:
# Get max item count per transaction to set K.
df = df[['order_id','shopping_cart']]
item_count = df.shopping_cart.apply(ast.literal_eval)
df['item_count'] = item_count
max = 0

for item in item_count:
  if(len(item) > max):
    max = len(item)
print(max)

4


<ipython-input-58-73544c82cf15>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['item_count'] = item_count


In [59]:
# Ignore shopping cart item count.
updated_shopping_cart = df.shopping_cart.apply(ast.literal_eval).apply(lambda x: [item[0] for item in x])
df['shopping_cart'] = updated_shopping_cart
df

,order_id,shopping_cart,item_count
0,ORD182494,"[Lucent 330S, Thunder line, iStream, pearTV]","[(Lucent 330S, 1), (Thunder line, 2), (iStream..."
1,ORD395518,"[Thunder line, Universe Note]","[(Thunder line, 1), (Universe Note, 2)]"
2,ORD494479,"[Thunder line, pearTV]","[(Thunder line, 1), (pearTV, 2)]"
3,ORD019224,"[Universe Note, Alcon 10, Olivia x460, iAssist...","[(Universe Note, 1), (Alcon 10, 2), (Olivia x4..."
4,ORD104032,"[Universe Note, Olivia x460, iStream, Toshika ...","[(Universe Note, 1), (Olivia x460, 1), (iStrea..."
...,...,...,...
495,ORD475510,"[Alcon 10, iAssist Line]","[(Alcon 10, 2), (iAssist Line, 2)]"
496,ORD086060,"[pearTV, Candle Inferno, Toshika 750]","[(pearTV, 2), (Candle Inferno, 2), (Toshika 75..."
497,ORD079320,"[pearTV, Alcon 10, Lucent 330S]","[(pearTV, 1), (Alcon 10, 1), (Lucent 330S, 1)]"
498,ORD026546,"[iStream, Candle Inferno, iAssist Line]","[(iStream, 1), (Candle Inferno, 1), (iAssist L..."


In [60]:
# Data cleaning
df['shopping_cart'] = [[item.lower() for item in shopping_cart] for shopping_cart in df['shopping_cart']]
df['shopping_cart']

0           [lucent 330s, thunder line, istream, peartv]
1                          [thunder line, universe note]
2                                 [thunder line, peartv]
3      [universe note, alcon 10, olivia x460, iassist...
4      [universe note, olivia x460, istream, toshika ...
                             ...                        
495                             [alcon 10, iassist line]
496                [peartv, candle inferno, toshika 750]
497                      [peartv, alcon 10, lucent 330s]
498              [istream, candle inferno, iassist line]
499                     [istream, toshika 750, alcon 10]
Name: shopping_cart, Length: 500, dtype: object

In [61]:
# shopping cart to dictionary. 
from collections import Counter
item_counter = Counter()
for shopping_cart in df['shopping_cart']:
    item_counter.update(shopping_cart)
original_shopping_cart_dictionary = dict(item_counter)
#shopping_cart_dictionary
# convert keys to tuples.
original_shopping_cart_dictionary = {tuple([key]): value for key, value in original_shopping_cart_dictionary.items()}
original_shopping_cart_dictionary

{('lucent 330s',): 164,
 ('thunder line',): 159,
 ('istream',): 145,
 ('peartv',): 154,
 ('universe note',): 152,
 ('alcon 10',): 143,
 ('olivia x460',): 154,
 ('iassist line',): 139,
 ('toshika 750',): 143,
 ('candle inferno',): 155}

In [ ]:
# Test fuction to get all individual items of the current C group in the algorithm
test = {('item1', 'item2'): 2, ('item2', 'item3'): 3, ('item3', 'item4'): 4}
current_items = set()
for key in test:
  for item in key:
    current_items.add(item)

current_items = list(current_items)
current_items

In [65]:
# A priori's first iteration
k = 3
support = 16

C_dictionaries = []

original_shopping_cart_dictionary = {key: value for key, value in original_shopping_cart_dictionary.items() if value >= support}
original_shopping_cart_dictionary

C_dictionaries.append(original_shopping_cart_dictionary)
C_dictionaries

[{('lucent 330s',): 164,
  ('thunder line',): 159,
  ('istream',): 145,
  ('peartv',): 154,
  ('universe note',): 152,
  ('alcon 10',): 143,
  ('olivia x460',): 154,
  ('iassist line',): 139,
  ('toshika 750',): 143,
  ('candle inferno',): 155}]

In [66]:
# A priori algorithm
shopping_cart_dictionary = original_shopping_cart_dictionary
# reduce k because first iter was already executed.
k = k-1
for ki in range(k):
  new_shopping_cart_dictionary = {}
  
  # Create a list of all individual items in the current C group
  # these are the items which have not been pruned.
  current_items = set()
  for key in shopping_cart_dictionary:
    for item in key:
      current_items.add(item)
  current_items = list(current_items)
  current_items

  # Get C combinations.
  for key1, value1 in shopping_cart_dictionary.items():
    #found_key1 = False
    #for key2, value2 in shopping_cart_dictionary.items():
    for key2 in current_items: 

      # Verify existense of both keys in shopping cart
      if key2 not in key1:
        # key_1 iter1: (lucent 330s), 
        # key_1 iter2: (lucent 330s, thunder)
        for shopping_cart in df['shopping_cart']:
          
          key1_in_shopping_cart = False
          #key2_in_shopping_cart = False
          key2_in_shopping_cart = True
          
          # key1
          for key_item in key1:
            key1_in_shopping_cart = True
            if key_item not in shopping_cart:
              key1_in_shopping_cart = False
              
          # key2
          if key2 not in shopping_cart:
            key2_in_shopping_cart = False

          # Add new combination to new shopping cart
          if (key1_in_shopping_cart and key2_in_shopping_cart):
            new_key = key1 + (key2,)
            new_shopping_cart_dictionary[new_key] = 0

  shopping_cart_dictionary = new_shopping_cart_dictionary

# Checkpoint 1: ok
#shopping_cart_dictionary

  # Count combination appearances.
  for shopping_cart in df['shopping_cart']:
    for key in shopping_cart_dictionary.keys():
      if all(key_item in shopping_cart for key_item in key):
        shopping_cart_dictionary[key] += 1

# Checkpoint 2: ok
#shopping_cart_dictionary

  # Prune
  shopping_cart_dictionary = {key: value for key, value in shopping_cart_dictionary.items() if value >= support}
  C_dictionaries.append(shopping_cart_dictionary)

# Checkpoint 3: ok
shopping_cart_dictionary


{('thunder line', 'universe note', 'peartv'): 16,
 ('thunder line', 'peartv', 'universe note'): 16,
 ('peartv', 'universe note', 'thunder line'): 16,
 ('peartv', 'thunder line', 'universe note'): 16,
 ('universe note', 'peartv', 'thunder line'): 16,
 ('universe note', 'thunder line', 'peartv'): 16}

In [79]:
# Association rules:
import itertools
confidence_threshold = 0.3
association_rules = []
for key in shopping_cart_dictionary.keys():
  # get all item combinations (subsets) in current key.
  key_subsets = []
  for length in range(1, len(key)): 
    key_subsets.extend(list((itertools.combinations(key, length))))

    # calculate confidence of each subset
    for key_subset in key_subsets:
      current_key_support_count = C_dictionaries[len(key)-1][key]
      current_key_subset_support_count = C_dictionaries[len(key_subset)-1][key_subset]

      confidence = current_key_support_count / current_key_subset_support_count
      if confidence >= confidence_threshold:
        association_rules.append((key_subset,key))
association_rules

[(('thunder line', 'universe note'),
  ('thunder line', 'universe note', 'peartv')),
 (('thunder line', 'peartv'), ('thunder line', 'universe note', 'peartv')),
 (('universe note', 'peartv'), ('thunder line', 'universe note', 'peartv')),
 (('thunder line', 'peartv'), ('thunder line', 'peartv', 'universe note')),
 (('thunder line', 'universe note'),
  ('thunder line', 'peartv', 'universe note')),
 (('peartv', 'universe note'), ('thunder line', 'peartv', 'universe note')),
 (('peartv', 'universe note'), ('peartv', 'universe note', 'thunder line')),
 (('peartv', 'thunder line'), ('peartv', 'universe note', 'thunder line')),
 (('universe note', 'thunder line'),
  ('peartv', 'universe note', 'thunder line')),
 (('peartv', 'thunder line'), ('peartv', 'thunder line', 'universe note')),
 (('peartv', 'universe note'), ('peartv', 'thunder line', 'universe note')),
 (('thunder line', 'universe note'),
  ('peartv', 'thunder line', 'universe note')),
 (('universe note', 'peartv'), ('universe note',